# Depth anything - HuggingFace 🤗

In [1]:
from transformers import pipeline
import cv2
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image
import ipywidgets as widgets
from tempfile import NamedTemporaryFile
from pathlib import Path
# Setup the pipeline
pipe = pipeline(task="depth-estimation", model="LiheYoung/depth-anything-small-hf")



##  Threshold widget example

In [2]:
# Define the threshold slider
threshold_slider = widgets.IntSlider(
    value=70,
    min=0,
    max=255,
    step=1,
    description='Threshold:',
    continuous_update=False
)

image = cv2.cvtColor(cv2.imread("../assets/sel&poivre.jpg"), cv2.COLOR_BGR2RGB)
depth = np.array(pipe(Image.fromarray(image))["depth"])

# Define the update function
def update_image(threshold):
    # Apply the threshold to the depth map
    depth_mask = cv2.threshold(depth, int(threshold), 255, cv2.THRESH_BINARY)[1]
    
    masked_image = cv2.bitwise_and(image, image, mask=depth_mask)
    
    # Display the images
    fig, axes = plt.subplots(1, 4, figsize=(16, 4))
    axes[0].imshow(image)
    axes[0].set_title('Original Image')
    axes[1].imshow(depth_mask, cmap='gray')
    axes[1].set_title('Depth Map (Thresholded)')
    axes[2].imshow(masked_image)
    axes[2].set_title('Masked Image')
    axes[3].imshow(depth, cmap='inferno')
    axes[3].set_title('Original Depth Mask')

    plt.tight_layout()
    plt.show()

widgets.interact(update_image, threshold=threshold_slider)

interactive(children=(IntSlider(value=70, continuous_update=False, description='Threshold:', max=255), Output(…

<function __main__.update_image(threshold)>

## Exemple 3D

In [3]:
import open3d as o3d

image = cv2.cvtColor(cv2.imread("../assets/sel&poivre.jpg"), cv2.COLOR_BGR2RGB)
depth = np.array(pipe(Image.fromarray(image))["depth"])
depth = cv2.resize(depth, (image.shape[1], image.shape[0]))

image_width = image.shape[1]
image_height = image.shape[0]

image = o3d.io.read_image("../assets/sel&poivre.jpg")
# Windows is not playing well with NamedTemporaryFile, so we need to set the delete=False to prevent a bug
with NamedTemporaryFile(delete=False, suffix=".png") as file:
    cv2.imwrite(file.name, depth)
    depth_img = o3d.io.read_image(file.name)
    intrinsics = o3d.camera.PinholeCameraIntrinsic(width=image_width, height=image_height, cx=image_width / 2, cy=image_height / 2, fx=image_width / 2, fy=image_height / 2) 
    pc = o3d.geometry.PointCloud() 
    rgbd = o3d.geometry.RGBDImage.create_from_color_and_depth(image, depth_img, depth_scale=1, depth_trunc=10000, convert_rgb_to_intensity=False)
    pc = pc.create_from_rgbd_image(rgbd, intrinsics) 
    o3d.visualization.draw_geometries([pc])
Path(file.name).unlink()

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
